In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use("seaborn-v0_8-pastel")
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
#from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.preprocessing import LabelEncoder

In [1]:
train_df=pd.read_csv(r"C:\Users\dnyap\OneDrive\Desktop\MLProject\src\notebook\data\train.csv",index_col='id')
test_df=pd.read_csv(r"C:\Users\dnyap\OneDrive\Desktop\MLProject\src\notebook\data\test.csv",index_col='id')
t_var=list(test_df.columns)
t_var

['Marital status',
 'Application mode',
 'Application order',
 'Course',
 'Daytime/evening attendance',
 'Previous qualification',
 'Previous qualification (grade)',
 'Nacionality',
 "Mother's qualification",
 "Father's qualification",
 "Mother's occupation",
 "Father's occupation",
 'Admission grade',
 'Displaced',
 'Educational special needs',
 'Debtor',
 'Tuition fees up to date',
 'Gender',
 'Scholarship holder',
 'Age at enrollment',
 'International',
 'Curricular units 1st sem (credited)',
 'Curricular units 1st sem (enrolled)',
 'Curricular units 1st sem (evaluations)',
 'Curricular units 1st sem (approved)',
 'Curricular units 1st sem (grade)',
 'Curricular units 1st sem (without evaluations)',
 'Curricular units 2nd sem (credited)',
 'Curricular units 2nd sem (enrolled)',
 'Curricular units 2nd sem (evaluations)',
 'Curricular units 2nd sem (approved)',
 'Curricular units 2nd sem (grade)',
 'Curricular units 2nd sem (without evaluations)',
 'Unemployment rate',
 'Inflation rat

In [2]:
target_category=['dropout', 'enrolled', 'graduate']
label_encoder=LabelEncoder()

train_df['Target']=label_encoder.fit_transform(train_df['Target'])

In [3]:
x_train=train_df[t_var]
y_train=train_df['Target']
x_test=test_df[t_var]

In [4]:
x_train

,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
id,,,,,,,,,,,,,,,,,,,,,
0,1,1,1,9238,1,1,126.0,1,1,19,...,0,0,6,7,6,12.428571,0,11.1,0.6,2.02
1,1,17,1,9238,1,1,125.0,1,19,19,...,0,0,6,9,0,0.000000,0,11.1,0.6,2.02
2,1,17,2,9254,1,1,137.0,1,3,19,...,0,0,6,0,0,0.000000,0,16.2,0.3,-0.92
3,1,1,3,9500,1,1,131.0,1,19,3,...,0,0,8,11,7,12.820000,0,11.1,0.6,2.02
4,1,1,2,9500,1,1,132.0,1,19,37,...,0,0,7,12,6,12.933333,0,7.6,2.6,0.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76513,1,17,1,9254,1,1,121.0,1,19,1,...,0,0,6,8,5,10.600000,0,13.9,-0.3,0.79
76514,1,1,6,9254,1,1,125.0,1,1,38,...,0,0,6,9,6,13.875000,0,9.4,-0.8,-3.12
76515,5,17,1,9085,1,1,138.0,1,37,37,...,2,0,5,8,5,11.400000,1,9.4,-0.8,-3.12


In [5]:
y_train

id
0        2
1        0
2        0
3        1
4        2
        ..
76513    2
76514    2
76515    1
76516    0
76517    2
Name: Target, Length: 76518, dtype: int64

In [6]:
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_square=r2_score(true,predicted)
    return mae,rmse,r2_square

In [7]:
        
"""
    Performs K-Fold cross-validation for a given model, returns the last model and average validation accuracy.

    Parameters:
        model: Machine learning model class (e.g., RandomForestClassifier)
        X_train: Training feature dataset
        y_train: Training target dataset
        params: Dictionary of parameters to initialize the model
        n_splits: Number of folds for cross-validation (default: 10)

    Returns:
        last_model: The last trained model instance
        average_val_accuracy: Average validation accuracy over all folds
"""

def cross_validation_model(model,x_train,y_train,params,n_splits=12):
    
    #intializing variables
    cross_v=KFold(n_splits=n_splits,shuffle=True,random_state=42)
    val_scores=[]


    #cross validation loop
    for fold, (train_ind,valid_ind) in enumerate(cross_v.split(x_train)):
      #data splitting
      x_fold_train=x_train.iloc[train_ind]
      y_fold_train=y_train.iloc[train_ind]
      x_val=x_train.iloc[valid_ind]
      y_val=y_train.iloc[valid_ind]

      #model initialization and training
      clf=model(**params)
      clf.fit(x_fold_train,y_fold_train)
      
      #predict and evaluate
      y_pred_trn=clf.predict(x_fold_train)
      y_pred_val=clf.predict(x_val)
      train_acc=accuracy_score(y_fold_train,y_pred_trn)
      val_acc=accuracy_score(y_val,y_pred_val)
      print(f"Fold: {fold}, Training Accuracy:{train_acc:.5f}, Val Accuracy:{val_acc:.5f}")
      print("*"*100)

      #accumulate validation scores
      val_scores.append(val_acc)

    avg_val_acc=np.mean(val_scores)
    print("Average Validation Accuracy:", avg_val_acc)

    return clf, avg_val_acc

RANDOM FOREST CLASSIFIER

In [8]:
print("Random Forest Classificcation Results: \n")
rf_model,rf_mean_acc=cross_validation_model(RandomForestClassifier,x_train,y_train,params={})

Random Forest Classificcation Results: 

Fold: 0, Training Accuracy:0.99996, Val Accuracy:0.81998
****************************************************************************************************
Fold: 1, Training Accuracy:0.99997, Val Accuracy:0.83535
****************************************************************************************************
Fold: 2, Training Accuracy:0.99996, Val Accuracy:0.83095
****************************************************************************************************
Fold: 3, Training Accuracy:0.99999, Val Accuracy:0.82939
****************************************************************************************************
Fold: 4, Training Accuracy:0.99997, Val Accuracy:0.83001
****************************************************************************************************
Fold: 5, Training Accuracy:1.00000, Val Accuracy:0.82139
****************************************************************************************************
Fold: 6, Tr

In [9]:
# Predict the test set and reverse the label encoding
rf_pred=rf_model.predict(x_test)
rf_pred_labels=label_encoder.inverse_transform(rf_pred)

#save the predictions to csv file
rf_result=pd.DataFrame(x_test.index)
rf_result['Target']=rf_pred_labels
rf_result.to_csv('resultRF.csv',index=False)
rf_result



,id,Target
0,76518,Dropout
1,76519,Graduate
2,76520,Graduate
3,76521,Graduate
4,76522,Enrolled
...,...,...
51007,127525,Dropout
51008,127526,Dropout
51009,127527,Dropout
51010,127528,Dropout


ADAPTIVE BOOSTING (ADABOOST)

In [10]:
print("ADABoost Cross validation Results:\n")
ada_model,ada_mean_acc=cross_validation_model(AdaBoostClassifier,x_train,y_train,params={})


ADABoost Cross validation Results:

Fold: 0, Training Accuracy:0.81999, Val Accuracy:0.81355
****************************************************************************************************
Fold: 1, Training Accuracy:0.81758, Val Accuracy:0.82374
****************************************************************************************************
Fold: 2, Training Accuracy:0.81928, Val Accuracy:0.82751
****************************************************************************************************
Fold: 3, Training Accuracy:0.81894, Val Accuracy:0.82609
****************************************************************************************************
Fold: 4, Training Accuracy:0.81904, Val Accuracy:0.82233
****************************************************************************************************
Fold: 5, Training Accuracy:0.82001, Val Accuracy:0.81559
****************************************************************************************************
Fold: 6, Trainin

In [11]:
# Predict the test set and reverse the label encoding
ada_pred=ada_model.predict(x_test)
ada_pred_labels=label_encoder.inverse_transform(ada_pred)

# Save the predictions to a CSV file
ada_result = pd.DataFrame(x_test.index)
ada_result['Target'] = ada_pred_labels
ada_result.to_csv('resultADA.csv', index=False)
ada_result

,id,Target
0,76518,Dropout
1,76519,Graduate
2,76520,Graduate
3,76521,Enrolled
4,76522,Dropout
...,...,...
51007,127525,Dropout
51008,127526,Dropout
51009,127527,Dropout
51010,127528,Dropout


Gradient Boosting

In [12]:
print("Gradient Boosting Cross Validation Results: \n")
gb_model,gb_mean_acc=cross_validation_model(GradientBoostingClassifier,x_train,y_train,params={})

Gradient Boosting Cross Validation Results: 

Fold: 0, Training Accuracy:0.83056, Val Accuracy:0.82531
****************************************************************************************************
Fold: 1, Training Accuracy:0.82960, Val Accuracy:0.83472
****************************************************************************************************
Fold: 2, Training Accuracy:0.82976, Val Accuracy:0.83550
****************************************************************************************************
Fold: 3, Training Accuracy:0.83023, Val Accuracy:0.82954
****************************************************************************************************
Fold: 4, Training Accuracy:0.82990, Val Accuracy:0.83205
****************************************************************************************************
Fold: 5, Training Accuracy:0.83131, Val Accuracy:0.82453
****************************************************************************************************
Fold: 

In [13]:
gb_pred=gb_model.predict(x_test)
gb_pred_labels=label_encoder.inverse_transform(gb_pred)

# Save the predictions to a CSV file
gb_result = pd.DataFrame(x_test.index)
gb_result['Target'] = gb_pred_labels
gb_result.to_csv('resultGB.csv', index=False)
gb_result

,id,Target
0,76518,Dropout
1,76519,Graduate
2,76520,Graduate
3,76521,Graduate
4,76522,Enrolled
...,...,...
51007,127525,Dropout
51008,127526,Dropout
51009,127527,Dropout
51010,127528,Dropout


EXTREME GRADIENT BOSSTING (XGBoost)

In [14]:
print("XGBoost Cross Validation Results:\n")
xgb_model,xgb_mean_acc=cross_validation_model(XGBClassifier,x_train,y_train,params={})

XGBoost Cross Validation Results:

Fold: 0, Training Accuracy:0.87793, Val Accuracy:0.83080
****************************************************************************************************
Fold: 1, Training Accuracy:0.87839, Val Accuracy:0.83613
****************************************************************************************************
Fold: 2, Training Accuracy:0.87813, Val Accuracy:0.83676
****************************************************************************************************
Fold: 3, Training Accuracy:0.87819, Val Accuracy:0.83284
****************************************************************************************************
Fold: 4, Training Accuracy:0.87856, Val Accuracy:0.83472
****************************************************************************************************
Fold: 5, Training Accuracy:0.87954, Val Accuracy:0.82625
****************************************************************************************************
Fold: 6, Training

In [15]:
xgb_pred=xgb_model.predict(x_test)
xgb_pred_labels=label_encoder.inverse_transform(xgb_pred)

# Save the predictions to a CSV file
xgb_result = pd.DataFrame(x_test.index)
xgb_result['Target'] = xgb_pred_labels
xgb_result.to_csv('resultXGB.csv', index=False)
xgb_result

,id,Target
0,76518,Dropout
1,76519,Graduate
2,76520,Graduate
3,76521,Enrolled
4,76522,Enrolled
...,...,...
51007,127525,Dropout
51008,127526,Dropout
51009,127527,Dropout
51010,127528,Dropout


LIGHT GRADIENT BOOSTING MACHINE (LightGBM)

In [16]:
print('LightGBM Cross-Validation Results:\n')
lgb_model, lgb_mean_acc = cross_validation_model(LGBMClassifier, x_train, y_train,params={})

LightGBM Cross-Validation Results:

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003860 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1381
[LightGBM] [Info] Number of data points in the train set: 70141, number of used features: 36
[LightGBM] [Info] Start training from score -1.106528
[LightGBM] [Info] Start training from score -1.634865
[LightGBM] [Info] Start training from score -0.745881
Fold: 0, Training Accuracy:0.84893, Val Accuracy:0.82798
****************************************************************************************************
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003167 seconds.
You can set `force_row_wise

In [17]:
# Predict the test set and reverse the label encoding
lgb_pred = lgb_model.predict(x_test)
lgb_pred_labels = label_encoder.inverse_transform(lgb_pred)

# Save the predictions to a CSV file
lgb_result = pd.DataFrame(x_test.index)
lgb_result['Target'] = lgb_pred_labels
lgb_result.to_csv('resultLGBM.csv', index=False)
lgb_result

,id,Target
0,76518,Dropout
1,76519,Graduate
2,76520,Graduate
3,76521,Graduate
4,76522,Enrolled
...,...,...
51007,127525,Dropout
51008,127526,Dropout
51009,127527,Dropout
51010,127528,Dropout


In [ ]:
'''cat_params = {
    'verbose': 0,                       # Silent mode
}

print('CatBoost Cross-Validation Results:\n')
cat_model, cat_mean_accuracy = cross_validation_model(CatBoostClassifier, x_train, y_train, cat_params)'''

In [ ]:
'''# Predict the test set and reverse the label encoding
cat_pred = cat_model.predict(x_test)
cat_pred_labels = label_encoder.inverse_transform(cat_pred)

# Save the predictions to a CSV file
cat_result = pd.DataFrame(x_test.index)
cat_result['Target'] = cat_pred_labels
cat_result.to_csv('resultCAT.csv', index=False)
cat_result'''

In [18]:
accuracy=pd.DataFrame({'Model': ['Random Forest', 'AdaBoost', 'Gradient Boosting', 'XGBoost', 'LightGBM'],
    'Score': [rf_mean_acc, ada_mean_acc, gb_mean_acc,
              xgb_mean_acc, lgb_mean_acc]})

In [19]:
accuracy_sorted = accuracy.sort_values(by='Score', ascending=False)
accuracy_sorted

,Model,Score
4,LightGBM,0.830889
3,XGBoost,0.830654
2,Gradient Boosting,0.827609
0,Random Forest,0.825989
1,AdaBoost,0.819049
